In [1]:
import pickle
from collections import Counter
from multiprocessing.pool import Pool

import numpy as np
import pandas as pd

from countries import get_countries

In [2]:
with open("data/actors.pkl", "rb") as f:
    df = pd.DataFrame(sorted(pickle.load(f)), columns=["nconst"])

def parallelize_dataframe(df, func, n_cores=12):
    df_split = np.array_split(df, n_cores)
    pool = Pool(n_cores)
    df = pd.concat(pool.map(func, df_split))
    pool.close()
    pool.join()
    return df

In [ ]:
df = parallelize_dataframe(df, get_countries)

In [ ]:
df.to_pickle("data/birth_countries.pkl")

In [ ]:
df_ = pd.read_pickle("data/birth_countries.pkl")
# pd.set_option("display.max_rows", None)
# print(df_)

countries = Counter(df_["birth_country"].values).most_common()
for i, country in enumerate(countries):
    print(i, country)